In [3]:
import os, sys
import numpy as np
import tensorflow as tf
src_path = os.path.join('..', 'src')
if src_path not in sys.path:
    sys.path.insert(0, src_path)
from W_Preproc import Weekly_Preprocessor as WP
from NumTForm import Transformer

In [4]:
'''
Create a generator from a lists of preprocessors
Batch size represents the number of weeks, not the number of
examples. The number of examples is much larger than the number of
weeks. The generator randomizes over the given wps in hopes of generalizing
over different periods of time
'''
def stochastic_gen(wp, weeks_in_batch):
    num_weeks = lambda wp: (wp.end_year - wp.start_year + 1) * 52
    rand_week = lambda num_weeks: int((np.random.random() * num_weeks) + 1)
    n_examples = 0
    while True:
        n_examples = 0
        xs = []
        ys = []
        while n_examples < weeks_in_batch:
            wp.cur_week = rand_week(num_weeks(wp))
            result = wp.get_next_week()
            if result is not None:
                x, y, x_names, prices, companies, b_date, s_date, cur_week = result
                xs.append(x)
                ys.append(y[:, None])
                n_examples += 1
        yield np.concatenate(xs, axis=0), np.concatenate(ys, axis=0)[:, 0], x_names

In [5]:
train = WP(40, 1970, 1995, binary=True)
val = WP(40, 1996, 2005, binary=True)
test = WP(40, 2010, 2021, binary=True)

In [7]:
data_generator = stochastic_gen(train, 20)
val_generator = stochastic_gen(val, 6)
cur_x, cur_y, x_names = data_generator.__next__()
val_x, val_y, _ = val_generator.__next__()

class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super(CustomSchedule, self).__init__()
        self.d_model = d_model
        self.d_model = tf.cast(self.d_model, tf.float32)

        self.warmup_steps = warmup_steps

    def __call__(self, step):
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)
        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)


num_layers = 6
d_model = 128
num_heads = 8
dff = 512

tform = Transformer(
            num_layers = num_layers,
            d_model = d_model,
            num_heads = num_heads,
            dff = dff)

learning_rate = CustomSchedule(d_model)
tform.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, 
                                            beta_2=0.98, epsilon=1e-9),
        loss=tf.keras.losses.MeanSquaredError())

for i in range(5):
    tform.fit(cur_x, cur_y, epochs=20, batch_size=128, validation_data=(val_x, val_y))
    pred = model.predict(val_x)
    print(i, pred.std())
    del cur_x
    del cur_y
    del val_x
    del val_y
    cur_x, cur_y, _ = data_generator.__next__()
    val_x, val_y, _ = val_generator.__next__()

Epoch 1/20


InternalError:  Attempting to perform BLAS operation using StreamExecutor without BLAS support
	 [[node transformer_1/encoder_1/sequential_7/dense_39/Tensordot/MatMul
 (defined at /opt/anaconda3/lib/python3.8/site-packages/keras/layers/core/dense.py:202)
]] [Op:__inference_train_function_23996]

Errors may have originated from an input operation.
Input Source operations connected to node transformer_1/encoder_1/sequential_7/dense_39/Tensordot/MatMul:
In[0] transformer_1/encoder_1/sequential_7/dense_39/Tensordot/Reshape:	
In[1] transformer_1/encoder_1/sequential_7/dense_39/Tensordot/ReadVariableOp:

Operation defined at: (most recent call last)
>>>   File "/opt/anaconda3/lib/python3.8/runpy.py", line 194, in _run_module_as_main
>>>     return _run_code(code, main_globals, None,
>>> 
>>>   File "/opt/anaconda3/lib/python3.8/runpy.py", line 87, in _run_code
>>>     exec(code, run_globals)
>>> 
>>>   File "/opt/anaconda3/lib/python3.8/site-packages/ipykernel_launcher.py", line 16, in <modu

In [11]:
data_generator = stochastic_gen(train, 20)
cur_x, _, _ = data_generator.__next__()
model.predict(cur_x[0:10])

array([[0.57250744],
       [0.5725075 ],
       [0.57250774],
       [0.57250744],
       [0.57250774],
       [0.5725075 ],
       [0.5725076 ],
       [0.5725077 ],
       [0.5725076 ],
       [0.5725075 ]], dtype=float32)

In [9]:
data_generator

<generator object stochastic_gen at 0x7f9d7217f3c0>